In [78]:
%pip install --upgrade pip
%pip install patool

Note: you may need to restart the kernel to use updated packages.



In [79]:
import pandas as pd
import os
from pathlib import Path
import json

pd.set_option("display.max_columns", None)

In [80]:
sample_csv_path = "Ibn-Alhaytham-Almousoaa.csv"
data_json_path = "products_data.json"
main_category_table = {
    "Ibn Alhaytham - backup": "Ibn Alhaytham, ",
    "Unipharma": "Unipharma, ",
    "Crystal Line": "Crystal Line, ",
    "HoneyCare": "HoneyCare, ",
    "Kabour": "Kabour Group, ",
    "linover": "Linover, ",
    "Paramedica": "Paramedica, "
}
folder_list = list(main_category_table.keys())

dealing with the csv file


In [81]:
sample_data = pd.read_csv(sample_csv_path)
sample_data

,ID,Type,SKU,Name,Published,Is featured?,Visibility in catalog,Short description,Description,Date sale price starts,Date sale price ends,Tax status,Tax class,In stock?,Stock,Low stock amount,Backorders allowed?,Sold individually?,Weight (kg),Length (cm),Width (cm),Height (cm),Allow customer reviews?,Purchase note,Sale price,Regular price,Categories,Tags,Shipping class,Images,Download limit,Download expiry days,Parent,Grouped products,Upsells,Cross-sells,External URL,Button text,Position,Attribute 1 name,Attribute 1 value(s),Attribute 1 visible,Attribute 1 global,Attribute 1 default,Attribute 2 name,Attribute 2 value(s),Attribute 2 visible,Attribute 2 global,Attribute 2 default,Meta: _eael_post_view_count,Meta: shopengine_product_views_count,Meta: _yoast_wpseo_primary_product_cat,Meta: _yoast_wpseo_estimated-reading-time-minutes,Meta: _yoast_wpseo_wordproof_timestamp,Meta: codesigner_quick_checkout
0,11323,variable,NaN,Loricstat,1,0,visible,NaN,NaN,NaN,NaN,taxable,NaN,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,"Ibn Alhaytham, Ibn Alhaytham > Urate Lowering ...",Urate Lowering Drugs,NaN,https://almousoaa.com/wp-content/uploads/2024/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Dosage form,Tablets,1.0,1,Tablets,Dosage Strength,"120, 40, 80",1.0,1.0,40.0,4.0,13.0,517.0,1.0,NaN,0.0
1,11325,variation,NaN,Loricstat,1,0,visible,NaN,Loricstat 40 mg\r\nTheraputic category: Urate ...,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,NaN,NaN,https://almousoaa.com/wp-content/uploads/2024/...,NaN,NaN,id:11323,NaN,NaN,NaN,NaN,NaN,1,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,40,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,11326,variation,NaN,Loricstat,1,0,visible,NaN,Loricstat 80 mg\r\nTheraputic category: Urate ...,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,NaN,NaN,https://almousoaa.com/wp-content/uploads/2024/...,NaN,NaN,id:11323,NaN,NaN,NaN,NaN,NaN,2,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,80,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,11327,variation,NaN,Loricstat,1,0,visible,NaN,Loricstat 120 mg\r\nTheraputic category: Urate...,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,1.0,NaN,NaN,NaN,https://almousoaa.com/wp-content/uploads/2024/...,NaN,NaN,id:11323,NaN,NaN,NaN,NaN,NaN,3,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,120,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,11329,variable,NaN,Loricstat ar,1,0,visible,NaN,arabic,NaN,NaN,taxable,NaN,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,1,NaN,NaN,NaN,ابن الهيثم,NaN,NaN,https://almousoaa.com/wp-content/uploads/2024/...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Dosage form,Tablets,1.0,1,Tablets,Dosage Strength,"120, 40, 80",1.0,1.0,40.0,10.0,13.0,555.0,1.0,NaN,0.0
5,11330,variation,NaN,Loricstat ar,1,0,visible,NaN,NaN,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,id:11329,NaN,NaN,NaN,NaN,NaN,1,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,40,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
6,11331,variation,NaN,Loricstat ar,1,0,visible,NaN,NaN,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,id:11329,NaN,NaN,NaN,NaN,NaN,2,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,80,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
7,11332,variation,NaN,Loricstat ar,1,0,visible,NaN,NaN,NaN,NaN,taxable,parent,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,id:11329,NaN,NaN,NaN,NaN,NaN,3,Dosage form,Tablets,NaN,1,NaN,Dosage Strength,120,NaN,1.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN
8,11333,simple,NaN,simple name,1,0,visible,NaN,simple disc,NaN,NaN,taxable,NaN,1,NaN,NaN,0,0,NaN,NaN,NaN,NaN,1,NaN,NaN,5555.0,Uncategorized,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0,Dosage form,Syrup,1.0,1,NaN,NaN,NaN,NaN,NaN,NaN,2.0,2.0,NaN,1.0,NaN,NaN


In [82]:
# setting the id as the index & removing it from the columns
df = sample_data.set_index(sample_data["ID"])

df.drop(labels="ID", axis=1, inplace=True)


# dropping cols that are all Nan

df.dropna(how="all", axis=1, inplace=True)


# dropping unnecessary meta columns

df.drop(
    labels=[
        "Meta: _eael_post_view_count",
        "Meta: shopengine_product_views_count",
        "Meta: _yoast_wpseo_primary_product_cat",
        "Meta: _yoast_wpseo_estimated-reading-time-minutes",
        "Meta: codesigner_quick_checkout",
    ],
    axis=1,
    inplace=True,
)


columns = df.columns
columns

Index(['Type', 'Name', 'Published', 'Is featured?', 'Visibility in catalog',
       'Description', 'Tax status', 'Tax class', 'In stock?',
       'Backorders allowed?', 'Sold individually?', 'Allow customer reviews?',
       'Regular price', 'Categories', 'Tags', 'Images', 'Parent', 'Position',
       'Attribute 1 name', 'Attribute 1 value(s)', 'Attribute 1 visible',
       'Attribute 1 global', 'Attribute 1 default', 'Attribute 2 name',
       'Attribute 2 value(s)', 'Attribute 2 visible', 'Attribute 2 global',
       'Attribute 2 default'],
      dtype='object')

In [83]:
# this is just for reference
df.loc[11323]

Type                                                                variable
Name                                                               Loricstat
Published                                                                  1
Is featured?                                                               0
Visibility in catalog                                                visible
Description                                                              NaN
Tax status                                                           taxable
Tax class                                                                NaN
In stock?                                                                  1
Backorders allowed?                                                        0
Sold individually?                                                         0
Allow customer reviews?                                                    1
Regular price                                                            NaN

In [84]:
# this is how we can add rows to our data
import numpy as np


ex_drug = (
    pd.Series(
        [
            "variable",
            "Loricstat",
            1,
            0,
            "visible",
            np.nan,
            "taxable",
            np.nan,
            1,
            0,
            0,
            1,
            np.nan,
            "Ibn Alhaytham, Ibn Alhaytham > Urate Lowering Drugs",
            "Urate Lowering Drugs",
            "img",
            np.nan,
            0,
            "Dosage form",
            "Tablets",
            1.0,
            1,
            "Tablets",
            "Dosage Strength",
            "120, 40, 80",
            1.0,
            1.0,
            40.0,
        ],
        index=columns,
        name=55,
    )
    .to_frame()
    .T
)

df = pd.concat([df, ex_drug])

df.drop_duplicates(inplace=True)  # beware cause the concat function is stackable
df.loc[55].values

array(['variable', 'Loricstat', 1, 0, 'visible', nan, 'taxable', nan, 1,
       0, 0, 1, nan,
       'Ibn Alhaytham, Ibn Alhaytham > Urate Lowering Drugs',
       'Urate Lowering Drugs', 'img', nan, 0, 'Dosage form', 'Tablets',
       1.0, 1, 'Tablets', 'Dosage Strength', '120, 40, 80', 1.0, 1.0,
       40.0], dtype=object)

In [85]:
can_be_automated = columns.drop(
    [
        "Attribute 1 value(s)",
        "Attribute 1 default",
        "Attribute 2 value(s)",
        "Attribute 2 default",
    ]
)

can_be_automated

Index(['Type', 'Name', 'Published', 'Is featured?', 'Visibility in catalog',
       'Description', 'Tax status', 'Tax class', 'In stock?',
       'Backorders allowed?', 'Sold individually?', 'Allow customer reviews?',
       'Regular price', 'Categories', 'Tags', 'Images', 'Parent', 'Position',
       'Attribute 1 name', 'Attribute 1 visible', 'Attribute 1 global',
       'Attribute 2 name', 'Attribute 2 visible', 'Attribute 2 global'],
      dtype='object')

In [86]:
# i can just leave them as nan and fill them later but who cares
# default values
published = 1
is_featured = 0
visibility_in_catalog = "visible"
tax_status = "taxable"
in_stock = 1
backorders_allowed = 0
sold_individually = 0
regular_price = 1
allow_costumer_reviews = 1
attribute_1_name = "Dosage form"
attribute_1_global = 1
attribute_2_name = "Dosage Strength"
attribute_2_global = 1

# things that need to be figured out:
# Name, Tax class, Type, Description, Categories, Parent, Position, Attribute 1 visible, Attribute 2 visible

# check how deep is the file in the img dir
variable = "variable"  # if its a variable then create and save id
variation = "variation"  # if its a variable create a variation for each one with the parent as the above id
simple = "simple"  # if simple just skip

attribute_1_visible = 1  # if its a variation then Nan else its 1
attribute_2_visible = 1  # same

now dealing with the csv file


In [87]:
# the json file
json_data = json.load(open(data_json_path))
json_data

{'Acnecid': {'Acnecid 10%': ['Acnecid 10% Gel\nTherapeutic category: Anti Acne\nActive constituents: Each 100 gram of Acnecid gel contains: 10 g benzoyl peroxide\nDescription:\nDosage form: Gel\nDosage strength:\nHow is supplied: Acnecid gel 10%: Tube of 30 g gel',
   'https://almousoaa.com/wp-content/uploads/2024/06/Acnecid-10.jpg'],
  'Acnecid 2.5%': ['Acnecid 2.5% Gel\nTherapeutic category: Anti Acne\nActive constituents: Each 100 gram of Acnecid gel contains: 2.5 g benzoyl peroxide\nDescription:\nDosage form: Gel\nDosage strength:\nHow is supplied: Acnecid gel 2.5% : Tubes of 30 g gel',
   'https://almousoaa.com/wp-content/uploads/2024/06/Acnecid-2.5.jpg'],
  'Acnecid 5% Gel': ['Acnecid 5 % Gel\nTherapeutic category: Anti Acne\nActive constituents: Each 100 gram of Acnecid gel contains: 5g benzoyl peroxide\nDescription:\nDosage form: Gel\nDosage strength:\nHow is supplied: Acnecid gel 5% : Tube of 30 g gel',
   'https://almousoaa.com/wp-content/uploads/2024/06/Acnecid-5-Gel.jpg'],


In [88]:
# products names without variations
products_names = list(json_data.keys())
products_names

['Acnecid',
 'Acnetrix',
 'Actaglim',
 'Alitensin',
 'Ambroxol',
 'Anti-Angina',
 'AntiAge',
 'Anxitin',
 'Aprevomit',
 'Argenor',
 'Arginine Ibn Al Haytham',
 'Artrofen',
 'Atorvastatin',
 'Azithromycin Ibn Al Haytham',
 'Azmixinal',
 'Baby Fort',
 'Be Smart',
 'Beta Active',
 'Bisoprol',
 'BonaVit',
 'Bradicore',
 'Brodimexine syrup',
 'Broncholysine',
 'Brophenil',
 'Budelar',
 'Calcium 200',
 'Ciprofloxacin',
 'Clarithromycin',
 'Coxibet',
 'Dapoxetine',
 'Dentalin Gel',
 'Depretin',
 'Dexlans',
 'Diabeset',
 'Diopiravin',
 'Energetic',
 'Fast Action',
 'Fenogesic',
 'Fexofenadine',
 'Floxalive',
 'Folevit',
 'Forderm cream',
 'Fosfomycin',
 'Gastrovit fawar',
 'Glucofer',
 'Glucosamine',
 'Glutaphos',
 'Grand Vita',
 'GreenTea Gum',
 'Harmonex',
 'Harmostar',
 'Hazi CaViD',
 'Healthy Pregna',
 'Indapamide SR',
 'Indomigrain',
 'Iron + Vitamins',
 'Iron-Folic-Zinc',
 'L-Sulpiride 25',
 'Lactisyr',
 'Lamic',
 'Leveramax',
 'Levo Histamine',
 'Loricstat',
 'Lornomax',
 'Mefenamic',
 

In [89]:
# init products csv
products = pd.DataFrame(columns=columns)
products

,Type,Name,Published,Is featured?,Visibility in catalog,Description,Tax status,Tax class,In stock?,Backorders allowed?,Sold individually?,Allow customer reviews?,Regular price,Categories,Tags,Images,Parent,Position,Attribute 1 name,Attribute 1 value(s),Attribute 1 visible,Attribute 1 global,Attribute 1 default,Attribute 2 name,Attribute 2 value(s),Attribute 2 visible,Attribute 2 global,Attribute 2 default


In [90]:
# Name, Tax class, Type, Description, Parent, Position, Attribute 1 visible, Attribute 2 visible

products = pd.DataFrame(columns=columns)

i = 0
id = 3000
while i < len(products_names):
    name = products_names[i]

    id += 1
    tax_class = None
    parent = None
    attribute_visible = 1
    position = 0
    description = None

    # getting the Type and Category
    product_type, category = get_product_type_and_category(name)

    if product_type == variable:
        length = len(json_data[name].keys())

        variable_images = []
        for j in range(length):
            variable_images.append(list(json_data[name].values())[j][1])
        img = ", ".join(variable_images)

        # create main variable product
        create_product(
            id,
            name,
            product_type,
            description,
            img,
            category,
            tax_class,
            parent,
            position,
            attribute_visible,
        )

        # iterate for every variation and create variation
        parent = id
        for j in range(length):
            product_type = variation
            position += 1
            id += 1
            tax_class = "parent"
            attribute_visible = None

            description, img = list(json_data[name].values())[j]

            create_product(
                id,
                name,
                product_type,
                description,
                img,
                category,
                tax_class,
                parent,
                position,
                attribute_visible,
            )
    elif product_type == simple:
        description, img = json_data[name]

        create_product(
            id,
            name,
            product_type,
            description,
            img,
            category,
            tax_class,
            parent,
            position,
            attribute_visible,
        )

    i += 1
    print(i)

1
2
3
4
5
6
7
8
9


10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
277
278
279
280
281
28

In [91]:
def find_folder(name, path):
    if type(path) == list:
        for p in path:
            finder = find_folder(name, p)
            if finder:
                return finder
    else:
        for root, dirs, files in os.walk(path):
            if name in dirs:
                return os.path.join(root, name)


def find_file(name, path):
    if type(path) == list:
        for p in path:
            finder = find_file(name, p)
            if finder:
                return finder
    else:

        for root, dirs, files in os.walk(path):

            if name + ".jpg" in files or name + ".png" in files:

                return os.path.join(root, name)


def get_global_categories(category):
    if category in [
        "Anti Acne",
        "Cardiovascular Drugs",
        "Respiratory Tract Drugs",
        "Amino Acids, Vitamins and Minerals",
        "Antidepressants",
        "Gastrointestinal Drugs",
        "Food Supplement",
        "Antiosteoporotic Drugs",
        "Erectile Dysfunction Therapy",
        "Antiallergics",
        "Dermatological Drugs",
        "Antimigraine Drugs",
        "Antiepileptics",
        "Urate Lowering Drugs",
        "Anti Diarrhea Drug",
        "Antiobesity Drugs",
        "Muscle Relaxants",
        "Otic Drugs",
        "Antispasmodics",
        "Urological Drugs",
        "Hyperuricemia and Gout Drugs",
        "Antihistamin",
        "Urinary Tract Drugs",
        "Worms Infestations Drugs",
        "Anti-Vertigo",
        "Neurological Drugs and Circulatory Stimulants",
        "Anti-Lice",
        "Anti-Obesity agents",
        "Antiedema",
        "Endocrine Drugs",
        "Gynecological Drugs",
        "NSAIDs",
        "Sexual Disorders Drugs",
        "Skin Care Creams",
        "Sunscreen Creams",
        "Hair Care",
        "Antiseptic Cream",
        "Deodorant",
        "Joint Pain Treatment",
        "Eye Health and Vision Care",
        "Osteoporosis Treatment",        
    ]:
        return category

    if category in [
        "Anti-inflammatory Drugs - Analgesics - Antipyretic",
        "Analgesics - Antipyretics",
    ]:
        return "Anti-inflammatory Drugs, Analgesics, Antipyretic"

    if category in [
        "Cholesterol Reducers - Hypolipidemic Drugs",
        "Hypolipidemic Drugs",
    ]:
        return "Cholesterol Reducers, Hypolipidemic Drugs"

    if category in [
        "Anti-infective Drugs - Antibacterial - Antiviral",
        "Anti-infection Drugs",
        "Antiviral Drugs",
    ]:
        return "Anti-infective Drugs, Antibacterial, Antiviral"

    if category in [
        "Anticoagulants and Antiplatelet",
        "Anti Platelets Aggregation Drugs",
    ]:
        return "Anticoagulants, Anti-Platelets Aggregation Drugs"

    if category == "Iron Supplements":
        return "Iron Supplements, Food Supplement"

    if category in ["Anti-infective Drugs - Antifungal", "Antifungal Drugs"]:
        return "Anti-infective Drugs, Antifungal"

    if category == "prostate diseases":
        return "Prostate Diseases Drugs"

    if category in [
        "Vitamins - Minerals",
        "Amino Acids, Vitamins and Minerals",
        "Amino Acids - Vitamins and Minerals",
    ]:
        return "Amino Acids - Vitamins and Minerals"

    if category == "Antidandruff":
        return "Anti Dandruff"

    if category in ["Antidiabetic Drugs", "Antidiabetics"]:
        return "Antidiabetics"

    if category in ["Dental and Topical Oral Preparations", "Dental Drugs"]:
        return "Dental and Topical Oral Preparations"

    if category in [
        "Mixtures",
        "Black Tea",
        "Green Tea",
        "Herbs",
        "Red Tea",
        "Powderd Milk",
    ]:
        return "Compo " + category

    if category == "White Labels":
        return "Compo White Label Tea"
    
    raise Exception(f"type {category} not found in golbal types")


def get_main_category(path):
    for k in folder_list:
        if path.count(k) > 0:
            return main_category_table[k]


def get_product_type_and_category(name: str):
    if file_path := find_folder(name, folder_list):
        product_type = variable
    elif file_path := find_file(name, folder_list):
        product_type = simple
    else:
        raise Exception("error getting the type for:" + name)

    raw_category = Path(file_path).parents[0].name
    category = get_main_category(file_path) + get_global_categories(raw_category)

    return product_type, category

In [92]:
def create_product(
    id,
    name,
    product_type,
    description,
    img,
    category,
    tax_class,
    parent,
    position,
    attribute_visible,
):
    s = (
        pd.Series(
            data=[
                product_type,
                name,
                published,
                is_featured,
                visibility_in_catalog,
                description,
                tax_status,
                tax_class,
                in_stock,
                backorders_allowed,
                sold_individually,
                allow_costumer_reviews,
                regular_price,
                category,
                "tags",
                img,
                parent,
                position,
                attribute_1_name,
                "Attribute 1 value(s)",
                attribute_visible,
                attribute_1_global,
                "Attribute 1 default",
                attribute_2_name,
                "Attribute 2 value(s)",
                attribute_visible,
                attribute_2_global,
                "Attribute 2 default",
            ],
            name=id,
            index=columns,
        )
        .to_frame()
        .T
    )
    global products
    products = pd.concat([products, s])

In [93]:
p = products[["Name", "Type", "Description"]]
print(p.count())
print()

print(p["Name"].nunique())
print()

t = []
for i in products.index:
    t.append(
        products.loc[i].get("Description")
        == products.loc[3001].get("Description")  # if empty
    )
products[t].get("Name").count()
# null desc: 61
# empty desc : 0

Name           603
Type           603
Description    507
dtype: int64

391



96

In [94]:
products[:3]

,Type,Name,Published,Is featured?,Visibility in catalog,Description,Tax status,Tax class,In stock?,Backorders allowed?,Sold individually?,Allow customer reviews?,Regular price,Categories,Tags,Images,Parent,Position,Attribute 1 name,Attribute 1 value(s),Attribute 1 visible,Attribute 1 global,Attribute 1 default,Attribute 2 name,Attribute 2 value(s),Attribute 2 visible,Attribute 2 global,Attribute 2 default
3001,variable,Acnecid,1,0,visible,None,taxable,None,1,0,0,1,1,"Ibn Alhaytham, Anti Acne",tags,https://almousoaa.com/wp-content/uploads/2024/...,None,0,Dosage form,Attribute 1 value(s),1,1,Attribute 1 default,Dosage Strength,Attribute 2 value(s),1,1,Attribute 2 default
3002,variation,Acnecid,1,0,visible,Acnecid 10% Gel\nTherapeutic category: Anti Ac...,taxable,parent,1,0,0,1,1,"Ibn Alhaytham, Anti Acne",tags,https://almousoaa.com/wp-content/uploads/2024/...,3001,1,Dosage form,Attribute 1 value(s),None,1,Attribute 1 default,Dosage Strength,Attribute 2 value(s),None,1,Attribute 2 default
3003,variation,Acnecid,1,0,visible,Acnecid 2.5% Gel\nTherapeutic category: Anti A...,taxable,parent,1,0,0,1,1,"Ibn Alhaytham, Anti Acne",tags,https://almousoaa.com/wp-content/uploads/2024/...,3001,2,Dosage form,Attribute 1 value(s),None,1,Attribute 1 default,Dosage Strength,Attribute 2 value(s),None,1,Attribute 2 default


In [95]:
final = products.reset_index(inplace=False, names="ID")
final[:2]

,ID,Type,Name,Published,Is featured?,Visibility in catalog,Description,Tax status,Tax class,In stock?,Backorders allowed?,Sold individually?,Allow customer reviews?,Regular price,Categories,Tags,Images,Parent,Position,Attribute 1 name,Attribute 1 value(s),Attribute 1 visible,Attribute 1 global,Attribute 1 default,Attribute 2 name,Attribute 2 value(s),Attribute 2 visible,Attribute 2 global,Attribute 2 default
0,3001,variable,Acnecid,1,0,visible,None,taxable,None,1,0,0,1,1,"Ibn Alhaytham, Anti Acne",tags,https://almousoaa.com/wp-content/uploads/2024/...,None,0,Dosage form,Attribute 1 value(s),1,1,Attribute 1 default,Dosage Strength,Attribute 2 value(s),1,1,Attribute 2 default
1,3002,variation,Acnecid,1,0,visible,Acnecid 10% Gel\nTherapeutic category: Anti Ac...,taxable,parent,1,0,0,1,1,"Ibn Alhaytham, Anti Acne",tags,https://almousoaa.com/wp-content/uploads/2024/...,3001,1,Dosage form,Attribute 1 value(s),None,1,Attribute 1 default,Dosage Strength,Attribute 2 value(s),None,1,Attribute 2 default


In [96]:
# saving the csv
products.to_csv("products.csv")
products.to_excel("products.xlsx")

In [97]:
products["Categories"].nunique()

83